In [1]:
import os
import sys
import re
import random
random.seed(0)

In [2]:
os.chdir("FinalProject/wav2vec-kenlm")

In [3]:
phrase_to_remove = "יעקב שולביץ"

In [4]:
with open('data/mesivta/ChavrutaShulevitz.output.txt') as f:
    lines = f.read().splitlines()

In [5]:
def clean_text(text):
    # Keep only Hebrew letters, punctuation marks (.,!?,;:), and spaces
    return re.sub(r'[^א-ת .,!?\n]', '', text)

clean_lines = [clean_text(line) for line in lines]

In [6]:
clean_lines[0]

' כל הזכויות שמורות  לרב יעקב שולביץ   חברותא  ברכות  בלי הערות  פרק ראשון  מאימתי  הקדמה '

In [7]:
def create_sections(lines):
    sections = []
    section=[]
    for line in lines:
        #if line starts with ###
        if line.startswith('###') and section!=[]:
            sections.append(section)
            section = []
            section.append(line)
        else:
            if line!='':
                section.append(line)
    sections.append(section)
    return sections
sections = create_sections(clean_lines)

In [ ]:
def split_sentences(section):
    # Split the section into sentences based on punctuation marks
    sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?|,|!)\s', ' '.join(section))
    # Remove trailing punctuation and extra spaces
    sentences = [sentence.strip(' .,!?') for sentence in sentences]
    return sentences

def combine_sentences(sections):
    combined_sentences = []
    for section in sections:
        sentences = split_sentences(section)
        combined_sentences.extend(sentences)
    return combined_sentences
combined_sentences=combine_sentences(sections)

In [ ]:
# def split_train_test(sections, percent_train=0.8):
#     random.shuffle(sections)
#     train = sections[:int(percent_train*len(sections))]
#     test = sections[int(percent_train*len(sections)):]
#     return {"train":train, "test":test}
# dataset=split_train_test(combined_sentences)

In [ ]:
def remove_specific_phrase(dataset, phrase):
    # Remove any row that contains the specified phrase
    clean_data = [section for section in dataset if phrase not in section]
    return clean_data

def remove_headers_and_white_space(dataset):
    #regex for all letters in between ### and ###
    clean_data=[re.sub(r'###.*?###', '', section) for section in dataset]
    clean_data=[re.sub(r' +', ' ', section) for section in clean_data]
    clean_data = [section.strip() for section in clean_data]
    return clean_data
dataset=remove_headers_and_white_space(combined_sentences)

# Apply the function to remove rows containing the specified phrase
dataset = remove_specific_phrase(dataset, phrase_to_remove)
# dataset['test']=remove_headers_and_white_space(dataset['test'])

In [ ]:
#write train and test to different files
with open('data/mesivta/mesivta_cleaned_for_kenlm.txt', 'w') as f:
    for item in dataset:
        f.write("%s\n" % item)
# with open(os.path.join(sys.path[0], 'data/mesivta/test.txt'), 'w') as f:
#     for item in dataset['test']:
#         f.write("%s\n" % item)